In [145]:
import numpy as np
import pandas as pd
#2 (Zhengxiang)
#JNJ Johnson & Johnson Healthcare
#PFE Pzer Inc. Pharmaceuticals
#UNH UnitedHealth Group Incorporated Healthcare
#XOM Exxon Mobil Corporation Energy
#CVX Chevron Corporation Energy


In [146]:
wholestock = pd.read_csv('Trading_Project_Data.csv',index_col=0)





In [147]:
wholestock.iloc[0]

Close               AAPL
Close.1             AMZN
Close.2               BA
Close.3              CAT
Close.4              CVX
Close.5            GOOGL
Close.6               GS
Close.7              JNJ
Close.8              JPM
Close.9               KO
Close.10             MCD
Close.11            MSFT
Close.12             NKE
Close.13            NVDA
Close.14             PFE
Close.15            SOFI
Close.16            TSLA
Close.17             UNH
Close.18             WMT
Close.19             XOM
Dividends           AAPL
Dividends.1         AMZN
Dividends.2           BA
Dividends.3          CAT
Dividends.4          CVX
Dividends.5        GOOGL
Dividends.6           GS
Dividends.7          JNJ
Dividends.8          JPM
Dividends.9           KO
Dividends.10         MCD
Dividends.11        MSFT
Dividends.12         NKE
Dividends.13        NVDA
Dividends.14         PFE
Dividends.15        SOFI
Dividends.16        TSLA
Dividends.17         UNH
Dividends.18         WMT
Dividends.19         XOM


In [148]:
def create_new_df(df, stock):
    ticker_row = df.iloc[0]
    matching_cols = ticker_row[ticker_row == stock].index.tolist()
    new_df = df[matching_cols]
    return new_df

jnj = create_new_df(wholestock, 'JNJ')
jnj.head()

    


,Close.7,Dividends.7,High.7,Low.7,Open.7,Stock Splits.7,Volume.7
Price,,,,,,,
Ticker,JNJ,JNJ,JNJ,JNJ,JNJ,JNJ,JNJ
Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22/1/2024,157.4842072,0,158.5601446,156.3888793,157.0286291,0,6824200
23/1/2024,154.905838,0,155.5843538,152.0948375,154.760447,0,9149800
24/1/2024,154.0819397,0,155.6716118,153.3064866,154.8476902,0,7247900


In [149]:
def clean_new_df(df):
    df = df.drop(jnj.index[0:2])
    df.index.name = 'Date'
    df['Close.7'] = pd.to_numeric(df['Close.7'], errors='coerce')
    df = df.dropna(subset=['Close.7'])

    df['log_return'] = np.log(df['Close.7'] / df['Close.7'].shift(1))
    df['log_return'] = df['log_return'].fillna(0)
    df['direction'] = np.sign(df['log_return'])
    df['lag_1'] = df['log_return'].shift(1)
    df['lag_2'] = df['log_return'].shift(2)
    df['lag_3'] = df['log_return'].shift(3)
    df.dropna(inplace=True)
    return df

jnj = clean_new_df(jnj)
print(jnj.head().to_string())




              Close.7 Dividends.7       High.7        Low.7       Open.7 Stock Splits.7 Volume.7  log_return  direction     lag_1     lag_2     lag_3
Date                                                                                                                                                 
25/1/2024  154.663498           0  154.7216543  153.6069509  154.1206853              0  5478900    0.003767        1.0 -0.005333 -0.016508  0.000000
26/1/2024  154.605347           0  155.8072995  154.4696435  155.5068002              0  5591900   -0.000376       -1.0  0.003767 -0.005333 -0.016508
29/1/2024  154.469650           0  155.1578687  154.0819312   154.130385              0  6483200   -0.000878       -1.0 -0.000376  0.003767 -0.005333
30/1/2024  153.897751           0  154.8573608  153.2773914  154.4599539              0  7200500   -0.003709       -1.0 -0.000878 -0.000376  0.003767
31/1/2024  154.023743           0  155.6909613   153.800805  155.4777114              0  8256800    

Algo 1:Deep Learning

In [150]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

model = MLPClassifier(hidden_layer_sizes = 2 * [75], max_iter = 2000,random_state=42)
train = jnj.loc['25/1/2024':'1/3/2024']
test = jnj.loc['4/3/2024':].copy()
x_train = train[['lag_1','lag_2','lag_3']]
y_train = train['direction']
x_test = test[['lag_1','lag_2','lag_3']]
y_test = test['direction']
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
test['pred'] = y_pred

test['strat'] = test['pred'] * test['log_return']
test['cumulative_strat'] = test['strat'].cumsum()
final_gain = np.exp(test['cumulative_strat'].iloc[-1])
print("Final Gain: ", final_gain)
test['market'] = test['log_return'].cumsum()
final_market_gain = np.exp(test['market'].iloc[-1])
print("Market Gain:", final_market_gain)




Final Gain:  0.9332562065972647
Market Gain: 0.9332562065972647


In [151]:
(test['pred'] == test['direction']).all()

np.False_

In [152]:
test['pred'].value_counts()

pred
1.0    220
Name: count, dtype: int64

In [144]:
train['direction'].value_counts()

direction
 1.0    13
-1.0    13
Name: count, dtype: int64

Algo 2: SVM

In [155]:
from sklearn.svm import SVC
model = SVC(kernel = 'linear')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
test['pred'] = y_pred
test['strat'] = test['pred'] * test['log_return']
test['cumulative_strat'] = test['strat'].cumsum()
final_gain = np.exp(test['cumulative_strat'].iloc[-1])
print("Final Gain: ", final_gain)
test['market'] = test['log_return'].cumsum()
final_market_gain = np.exp(test['market'].iloc[-1])
print("Market Gain:", final_market_gain)




Final Gain:  0.7787410075152574
Market Gain: 0.9332562065972647


Algo3: Decision Tree

In [172]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=10)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
test['pred'] = y_pred
test['strat'] = test['pred'] * test['log_return']
test['cumulative_strat'] = test['strat'].cumsum()
final_gain = np.exp(test['cumulative_strat'].iloc[-1])
print("Final Gain: ", final_gain)
test['market'] = test['log_return'].cumsum()
final_market_gain = np.exp(test['market'].iloc[-1])
print("Market Gain:", final_market_gain)


Final Gain:  0.9955418250760493
Market Gain: 0.9332562065972647


Algo 4:Logistic regression

In [163]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
test['pred'] = y_pred
test['strat'] = test['pred'] * test['log_return']
test['cumulative_strat'] = test['strat'].cumsum()
final_gain = np.exp(test['cumulative_strat'].iloc[-1])
print("Final Gain: ", final_gain)
test['market'] = test['log_return'].cumsum()
final_market_gain = np.exp(test['market'].iloc[-1])
print("Market Gain:", final_market_gain)


Final Gain:  0.8453321559726146
Market Gain: 0.9332562065972647


Conclusion: Decision tree is best